In [1]:
import librosa
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def extract_features(audio_path, sr=22050):
    """음성 파일로부터 MFCC 특징 추출"""
    audio, _ = librosa.load(audio_path, sr=sr)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

def register_voice(reference_audio_path):
    """등록된 사용자의 음성 특징 저장"""
    return extract_features(reference_audio_path)

def compare_voices(registered_features, new_audio_path):
    """새로운 음성을 등록된 음성과 비교하여 유사도 계산"""
    new_features = extract_features(new_audio_path)
    similarity = cosine_similarity([registered_features], [new_features])
    return similarity[0][0]  # 코사인 유사도 반환

if __name__ == "__main__":
    # 1. 자신의 목소리 등록 (ex: my_voice.wav)
    registered_features = register_voice("output.wav")

    # 2. 새로운 목소리와 비교 (ex: other_voice.wav)
    similarity_score = compare_voices(registered_features, "빨강.mp3")

    print(f"유사도 점수: {similarity_score:.2f}")

    if similarity_score > 0.75:
        print("두 목소리가 유사합니다.")
    else:
        print("두 목소리가 다릅니다.")


유사도 점수: 0.98
두 목소리가 유사합니다.


In [14]:
import librosa
import numpy as np
from scipy.spatial.distance import cdist
from fastdtw import fastdtw

def extract_mfcc(audio_path, sr=22050):
    """음성 파일에서 MFCC 특징 추출"""
    audio, _ = librosa.load(audio_path, sr=sr)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return mfccs.T  # 시간 축 기준으로 변환

def compute_dtw_distance(mfcc1, mfcc2):
    """DTW를 사용한 두 음성의 거리 계산"""
    distance, _ = fastdtw(mfcc1, mfcc2, dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    return distance

# 파일 경로 설정
audio1_path = "검정.mp3"
audio2_path = "output.wav"

# MFCC 특징 추출
mfcc1 = extract_mfcc(audio1_path)
mfcc2 = extract_mfcc(audio2_path)

# DTW 거리 계산
dtw_distance = compute_dtw_distance(mfcc1, mfcc2)

print(f"DTW 거리: {dtw_distance}")

# 유사도 계산 (거리가 작을수록 유사함)
similarity_score = np.exp(-dtw_distance / 1000)  # 거리값을 유사도로 변환
print(f"유사도 점수: {similarity_score:.2f}")


DTW 거리: 78795.04337430745
유사도 점수: 0.00


In [20]:
import librosa

# 파일 경로 설정
audio1_path = "빨강.mp3"
audio2_path = "output.wav"

# 음성 파일 정보 출력
audio1, sr1 = librosa.load(audio1_path, sr=None)  # 원본 SR 유지
audio2, sr2 = librosa.load(audio2_path, sr=None)  # 원본 SR 유지

print(f"빨강.mp3 - 샘플링 레이트: {sr1}, 길이(초): {len(audio1)/sr1}")
print(f"검정.mp3 - 샘플링 레이트: {sr2}, 길이(초): {len(audio2)/sr2}")


빨강.mp3 - 샘플링 레이트: 24000, 길이(초): 1.05
검정.mp3 - 샘플링 레이트: 16000, 길이(초): 4.992


In [21]:
audio1, _ = librosa.load(audio1_path, sr=22050)
audio2, _ = librosa.load(audio2_path, sr=22050)

In [22]:
min_length = min(len(audio1), len(audio2))
audio1 = audio1[:min_length]
audio2 = audio2[:min_length]


In [23]:
import librosa

# 파일 경로 설정
audio1_path = "빨강.mp3"
audio2_path = "output.wav"

# 음성 파일 정보 출력
audio1, sr1 = librosa.load(audio1_path, sr=None)  # 원본 SR 유지
audio2, sr2 = librosa.load(audio2_path, sr=None)  # 원본 SR 유지

print(f"빨강.mp3 - 샘플링 레이트: {sr1}, 길이(초): {len(audio1)/sr1}")
print(f"검정.mp3 - 샘플링 레이트: {sr2}, 길이(초): {len(audio2)/sr2}")

빨강.mp3 - 샘플링 레이트: 24000, 길이(초): 1.05
검정.mp3 - 샘플링 레이트: 16000, 길이(초): 4.992


In [18]:
import librosa
import numpy as np
from fastdtw import fastdtw

def extract_mfcc(audio, sr=22050):
    """오디오 데이터로부터 MFCC 추출"""
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return mfccs.T  # 시간 축 기준으로 변환

def compute_dtw_distance(mfcc1, mfcc2):
    """DTW 알고리즘을 이용한 두 음성 간의 거리 계산"""
    distance, _ = fastdtw(mfcc1, mfcc2, dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    return distance

# 음성 파일 로드 및 샘플링 레이트 맞추기
audio1, sr = librosa.load("빨강.mp3", sr=22050)
audio2, _ = librosa.load("output.wav", sr=22050)

# 음성 길이 맞추기
min_len = min(len(audio1), len(audio2))
audio1 = audio1[:min_len]
audio2 = audio2[:min_len]

# MFCC 특징 추출
mfcc1 = extract_mfcc(audio1, sr)
mfcc2 = extract_mfcc(audio2, sr)

# DTW 거리 계산 및 유사도 변환
dtw_distance = compute_dtw_distance(mfcc1, mfcc2)
similarity_score = np.exp(-dtw_distance / 1000)  # 거리에서 유사도로 변환

print(f"DTW 거리: {dtw_distance}")
print(f"유사도 점수: {similarity_score:.2f}")


DTW 거리: 32648.610815808177
유사도 점수: 0.00


In [2]:
import librosa
import numpy as np
from fastdtw import fastdtw

def extract_features(audio_path, sr=22050):
    """MFCC와 피치(F0)를 함께 추출"""
    audio, _ = librosa.load(audio_path, sr=sr)

    # 1. MFCC 추출
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfccs, axis=1)  # MFCC 평균 계산

    # 2. 피치(F0) 추출
    pitches, _ = librosa.core.piptrack(y=audio, sr=sr)
    pitch_mean = np.mean(pitches[pitches > 0])  # 0이 아닌 피치의 평균값

    # 3. 특징 벡터 결합 (MFCC + 피치)
    features = np.append(mfcc_mean, pitch_mean)
    return features

def compare_voices(audio1_path, audio2_path):
    """두 음성의 MFCC + 피치 기반 유사도 계산"""
    features1 = extract_features(audio1_path)
    features2 = extract_features(audio2_path)

    # DTW 알고리즘으로 유사도 계산
    distance, _ = fastdtw(features1.reshape(-1, 1), features2.reshape(-1, 1), 
                          dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    
    similarity_score = np.exp(-distance / 100)  # 거리값을 유사도로 변환
    return distance, similarity_score

# 파일 경로 설정
audio1_path = "빨강.mp3"
audio2_path = "검정.mp3"

# 두 음성 파일 비교
distance, similarity = compare_voices(audio1_path, audio2_path)

print(f"DTW 거리: {distance}")
print(f"유사도 점수: {similarity:.2f}")

if similarity > 0.75:
    print("두 목소리가 유사합니다.")
else:
    print("두 목소리가 다릅니다.")


DTW 거리: 251.28373324871063
유사도 점수: 0.08
두 목소리가 다릅니다.


In [38]:
import pyaudio
import wave
import numpy as np
import librosa
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

# 음성을 녹음하는 함수
def record_audio(filename, duration=5, rate=44100, channels=1):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=1024)
    
    print("Recording...")
    frames = []
    for _ in range(0, int(rate / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)
    
    print("Finished recording")
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

# 성별 분류 모델을 학습하고 저장하는 함수
def train_gender_classifier(model_path='gender_classifier.pkl'):
    # 데이터 준비 (여기서는 예시로 가상 데이터를 사용)
    # 실제로는 남성/여성 음성 데이터를 수집하여 사용해야 합니다.
    X = []
    y = []
    male_data_path = 'data/male'
    female_data_path = 'data/female'
    
    if not os.path.exists(male_data_path) or not os.path.exists(female_data_path):
        print(f"Error: Data directories '{male_data_path}' and '{female_data_path}' not found. Please provide the necessary data.")
        return
    
    for filename in os.listdir(male_data_path):
        filepath = os.path.join(male_data_path, filename)
        if os.path.isfile(filepath):
            audio, sr = librosa.load(filepath, sr=None)
            mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
            mfcc_mean = np.mean(mfcc, axis=1)
            X.append(mfcc_mean)
            y.append(1)  # 남성 레이블
    for filename in os.listdir(female_data_path):
        filepath = os.path.join(female_data_path, filename)
        if os.path.isfile(filepath):
            audio, sr = librosa.load(filepath, sr=None)
            mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
            mfcc_mean = np.mean(mfcc, axis=1)
            X.append(mfcc_mean)
            y.append(0)  # 여성 레이블
    
    X = np.array(X)
    y = np.array(y)
    
    # 학습/테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 모델 학습
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    model = SVC(kernel='linear')
    model.fit(X_train_scaled, y_train)
    
    # 모델 저장
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    
    # 테스트 및 정확도 출력
    X_test_scaled = scaler.transform(X_test)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model training complete. Accuracy: {accuracy * 100:.2f}%")

# 녹음한 파일을 이용해 목소리의 성별을 예측하는 함수
def predict_gender(filename, model_path='gender_classifier.pkl'):
    # 모델이 없으면 학습 후 저장
    if not os.path.exists(model_path):
        print("Model not found. Training new model...")
        train_gender_classifier(model_path)
        if not os.path.exists(model_path):
            print("Model training failed. Please provide the necessary data.")
            return
    
    # 녹음된 음성 파일을 로드하고 특징 추출
    audio, sr = librosa.load(filename, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    
    # 머신러닝 모델 로드
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # 데이터 스케일링 후 성별 예측
    scaler = StandardScaler()
    mfcc_mean_scaled = scaler.fit_transform([mfcc_mean])
    gender_prediction = model.predict(mfcc_mean_scaled)
    
    if gender_prediction[0] == 1:
        print("Predicted Gender: Male")
    else:
        print("Predicted Gender: Female")

if __name__ == "__main__":
    # 음성을 녹음하여 파일로 저장
    record_audio("recorded_voice.wav")
    
    # 녹음한 음성의 성별을 예측
    predict_gender("recorded_voice.wav")


Recording...
Finished recording
Predicted Gender: Male


In [97]:
import pyaudio
import wave
import numpy as np
import librosa
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

# 음성을 녹음하는 함수
def record_audio(filename, duration=5, rate=44100, channels=1):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=1024)
    
    print("Recording...")
    frames = []
    for _ in range(0, int(rate / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)
    
    print("Finished recording")
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

# 성별 분류 모델을 학습하고 저장하는 함수 (특징 데이터만 사용)
def train_gender_classifier_from_csv(csv_path, model_path='gender_classifier.pkl', scaler_path='scaler.pkl'):
    # CSV 파일로부터 데이터 로드
    if not os.path.exists(csv_path):
        print(f"Error: CSV file '{csv_path}' not found. Please provide the necessary data.")
        return
    
    data = pd.read_csv(csv_path)
    if 'label' not in data.columns:
        print("Error: CSV file must contain 'label' column.")
        return
    
    # 'label' 열 값을 숫자로 변환 (male -> 1, female -> 0)
    data['label'] = data['label'].map({'male': 1, 'female': 0})
    
    # 특징 데이터와 라벨 추출 (label 열을 제외한 나머지 열을 특징으로 사용)
    X = data.drop(columns=['label']).values
    y = data['label'].values
    
    if len(X) == 0:
        print("Error: No valid feature data found in the CSV.")
        return
    
    # 학습/테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 모델 학습
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = SVC(kernel='linear')
    model.fit(X_train_scaled, y_train)
    
    # 모델 및 스케일러 저장
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    
    # 테스트 및 정확도 출력
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model training complete. Accuracy: {accuracy * 100:.2f}%")

# 녹음한 파일을 이용해 목소리의 성별을 예측하는 함수
def predict_gender(filename, model_path='gender_classifier.pkl', scaler_path='scaler.pkl'):
    # 모델과 스케일러가 없으면 학습 후 저장
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        print("Model or scaler not found. Please train the model first using a CSV file.")
        return
    
    # 녹음된 음성 파일을 로드하고 특징 추출
    audio, sr = librosa.load(filename, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=21)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_mean = mfcc_mean.reshape(1, -1)  # reshape to match the scaler's expected input
    
    # 머신러닝 모델 및 스케일러 로드
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # 데이터 스케일링 후 성별 예측
    mfcc_mean_scaled = scaler.transform(mfcc_mean)
    gender_prediction = model.predict(mfcc_mean_scaled)
    
    if gender_prediction[0] == 1:
        print("Predicted Gender: Male")
    else:
        print("Predicted Gender: Female")

if __name__ == "__main__":
    # 음성을 녹음하여 파일로 저장
    record_audio("recorded_voice.wav")
    
    # CSV 파일에서 모델 학습 (사용자가 CSV 경로를 제공해야 함)
    train_gender_classifier_from_csv(r'C:\Projects\Python_basic\최종프로젝트\further_augmented_voice_data.csv')
    
    # 녹음한 음성의 성별을 예측
    predict_gender("recorded_voice.wav")


Recording...
Finished recording
Model training complete. Accuracy: 80.90%
Predicted Gender: Male


In [100]:
import pyaudio
import wave
import numpy as np
import librosa
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

# 음성을 파일로부터 로드하여 성별을 예측하는 함수
def predict_gender_from_file(filename, model_path='gender_classifier.pkl', scaler_path='scaler.pkl'):
    # 모델과 스케일러가 없으면 학습 후 저장
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        print("Model or scaler not found. Please train the model first using a CSV file.")
        return
    
    # 음성 파일을 로드하고 특징 추출
    audio, sr = librosa.load(filename, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=21)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_mean = mfcc_mean.reshape(1, -1)  # reshape to match the scaler's expected input
    
    # 머신러닝 모델 및 스케일러 로드
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # 데이터 스케일링 후 성별 예측
    mfcc_mean_scaled = scaler.transform(mfcc_mean)
    gender_prediction = model.predict(mfcc_mean_scaled)
    
    if gender_prediction[0] == 1:
        print("Predicted Gender: Male")
    else:
        print("Predicted Gender: Female")

# 성별 분류 모델을 학습하고 저장하는 함수 (특징 데이터만 사용)
def train_gender_classifier_from_csv(csv_path, model_path='gender_classifier.pkl', scaler_path='scaler.pkl'):
    # CSV 파일로부터 데이터 로드
    if not os.path.exists(csv_path):
        print(f"Error: CSV file '{csv_path}' not found. Please provide the necessary data.")
        return
    
    data = pd.read_csv(csv_path)
    if 'label' not in data.columns:
        print("Error: CSV file must contain 'label' column.")
        return
    
    # 'label' 열 값을 숫자로 변환 (male -> 0, female -> 1)
    data['label'] = data['label'].map({'male': 0, 'female': 1})
    
    # 특징 데이터와 라벨 추출 (label 열을 제외한 나머지 열을 특징으로 사용)
    X = data.drop(columns=['label']).values
    y = data['label'].values
    
    if len(X) == 0:
        print("Error: No valid feature data found in the CSV.")
        return
    
    # 학습/테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 모델 학습
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = SVC(kernel='linear')
    model.fit(X_train_scaled, y_train)
    
    # 모델 및 스케일러 저장
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    
    # 테스트 및 정확도 출력
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model training complete. Accuracy: {accuracy * 100:.2f}%")

if __name__ == "__main__":
    # CSV 파일에서 모델 학습 (사용자가 CSV 경로를 제공해야 함)
    train_gender_classifier_from_csv(r'C:\Projects\Python_basic\최종프로젝트\further_augmented_voice_data.csv')
    
    # 음성 파일의 성별을 예측
    predict_gender_from_file(r"C:\Projects\Python_basic\최종프로젝트\빨강.mp3")

Model training complete. Accuracy: 80.90%
Predicted Gender: Female


In [135]:
import pyaudio
import wave
import numpy as np
import librosa
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
import os

# 음성을 녹음하는 함수
def record_audio(filename, duration=5, rate=44100, channels=1):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=1024)
    
    print("Recording...")
    frames = []
    for _ in range(0, int(rate / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)
    
    print("Finished recording")
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

# 성별 분류 모델을 학습하고 저장하는 함수 (특징 데이터만 사용)
def train_gender_classifier_from_csv(csv_path, model_path='gender_classifier.pkl', scaler_path='scaler.pkl'):
    # CSV 파일로부터 데이터 로드
    if not os.path.exists(csv_path):
        print(f"Error: CSV file '{csv_path}' not found. Please provide the necessary data.")
        return
    
    data = pd.read_csv(csv_path)
    if 'label' not in data.columns:
        print("Error: CSV file must contain 'label' column.")
        return
    
    # 'label' 열 값을 숫자로 변환 (male -> 1, female -> 0)
    data['label'] = data['label'].map({'male': 1, 'female': 0})
    
    # 특징 데이터와 라벨 추출 (label 열을 제외한 나머지 열을 특징으로 사용)
    X = data.drop(columns=['label']).values
    y = data['label'].values
    
    if len(X) == 0:
        print("Error: No valid feature data found in the CSV.")
        return
    
    # 학습/테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 모델 학습
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    
    # 모델 및 스케일러 저장
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    
    # 테스트 및 정확도 출력
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model training complete. Accuracy: {accuracy * 100:.2f}%")

# 녹음한 파일을 이용해 목소리의 성별을 예측하는 함수
def predict_gender(filename, model_path='gender_classifier.pkl', scaler_path='scaler.pkl'):
    # 모델과 스케일러가 없으면 학습 후 저장
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        print("Model or scaler not found. Please train the model first using a CSV file.")
        return
    
    # 녹음된 음성 파일을 로드하고 특징 추출
    audio, sr = librosa.load(filename, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=21)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_mean = mfcc_mean.reshape(1, -1)  # reshape to match the scaler's expected input
    
    # 머신러닝 모델 및 스케일러 로드
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # 데이터 스케일링 후 성별 예측
    mfcc_mean_scaled = scaler.transform(mfcc_mean)
    gender_prediction = model.predict(mfcc_mean_scaled)
    
    if gender_prediction[0] == 1:
        print("Predicted Gender: Male")
    else:
        print("Predicted Gender: Female")

if __name__ == "__main__":
    # 음성을 녹음하여 파일로 저장
    record_audio("recorded_voice.wav")
    
    # CSV 파일에서 모델 학습 (사용자가 CSV 경로를 제공해야 함)
    train_gender_classifier_from_csv(r'C:\Projects\Python_basic\최종프로젝트\further_augmented_voice_data.csv')
    
    # 녹음한 음성의 성별을 예측
    predict_gender("recorded_voice.wav")


Recording...
Finished recording
Model training complete. Accuracy: 91.72%
Predicted Gender: Male


In [133]:
import pyaudio
import wave
import numpy as np
import librosa
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

# 음성을 파일로부터 로드하여 성별을 예측하는 함수
def predict_gender_from_file(filename, model_path='gender_classifier.pkl', scaler_path='scaler.pkl'):
    # 모델과 스케일러가 없으면 학습 후 저장
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        print("Model or scaler not found. Please train the model first using a CSV file.")
        return
    
    # 음성 파일을 로드하고 특징 추출
    audio, sr = librosa.load(filename, sr=None)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=21)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_mean = mfcc_mean.reshape(1, -1)  # reshape to match the scaler's expected input
    
    # 머신러닝 모델 및 스케일러 로드
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    # 데이터 스케일링 후 성별 예측
    mfcc_mean_scaled = scaler.transform(mfcc_mean)
    gender_prediction = model.predict(mfcc_mean_scaled)
    
    if gender_prediction[0] == 1:
        print("Predicted Gender: Male")
    else:
        print("Predicted Gender: Female")

# 성별 분류 모델을 학습하고 저장하는 함수 (특징 데이터만 사용)
def train_gender_classifier_from_csv(csv_path, model_path='gender_classifier.pkl', scaler_path='scaler.pkl'):
    # CSV 파일로부터 데이터 로드
    if not os.path.exists(csv_path):
        print(f"Error: CSV file '{csv_path}' not found. Please provide the necessary data.")
        return
    
    data = pd.read_csv(csv_path)
    if 'label' not in data.columns:
        print("Error: CSV file must contain 'label' column.")
        return
    
    # 'label' 열 값을 숫자로 변환 (male -> 0, female -> 1)
    data['label'] = data['label'].map({'male': 0, 'female': 1})
    
    # 특징 데이터와 라벨 추출 (label 열을 제외한 나머지 열을 특징으로 사용)
    X = data.drop(columns=['label']).values
    y = data['label'].values
    
    if len(X) == 0:
        print("Error: No valid feature data found in the CSV.")
        return
    
    # 학습/테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 모델 학습
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    
    # 모델 및 스케일러 저장
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    
    # 테스트 및 정확도 출력
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model training complete. Accuracy: {accuracy * 100:.2f}%")

if __name__ == "__main__":
    # CSV 파일에서 모델 학습 (사용자가 CSV 경로를 제공해야 함)
    train_gender_classifier_from_csv(r'C:\Projects\Python_basic\최종프로젝트\further_augmented_voice_data.csv')
    
    # 음성 파일의 성별을 예측
    predict_gender_from_file(r"C:\Projects\Python_basic\최종프로젝트\data\male\06_손목이_아파.wav")


Model training complete. Accuracy: 91.90%
Predicted Gender: Female


In [147]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import librosa
import numpy as np

# CSV 파일 불러오기
file_path = r'C:\Projects\Python_basic\최종프로젝트\Male and female Voice data creat by al arman ovi .csv'
voice_data = pd.read_csv(file_path)

# Step 1: 데이터 전처리
# 특성과 라벨을 분리
X = voice_data.drop(columns=['label'])
y = voice_data['label']

# 라벨을 남성(0), 여성(1)로 변환
y = y.map({'male': 0, 'female': 1})

# Step 2: 학습 및 테스트 데이터 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: RandomForestClassifier로 모델 학습
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 4: 테스트 데이터에 대한 예측 및 정확도 확인
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"모델의 정확도: {accuracy * 100:.2f}%")

# Step 5: 음성 파일로부터 학습한 데이터와 동일한 21개의 특징을 추출하는 함수
def extract_features_from_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)  # 음성 파일 로드
    features = []

    # 1. 평균 주파수 관련 값 (meanfreq)
    features.append(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)))

    # 2. 표준 편차 (sd) - 스펙트럼 중심의 표준 편차로 사용
    features.append(np.std(librosa.feature.spectral_centroid(y=y, sr=sr)))

    # 3. 중앙값 주파수 (median_freq) - 중앙값을 추정
    features.append(np.median(librosa.feature.spectral_centroid(y=y, sr=sr)))

    # 4. 첫 번째 사분위수 (Q25) - 스펙트럼 롤오프의 25%
    features.append(np.percentile(librosa.feature.spectral_rolloff(y=y, sr=sr), 25))

    # 5. 세 번째 사분위수 (Q75) - 스펙트럼 롤오프의 75%
    features.append(np.percentile(librosa.feature.spectral_rolloff(y=y, sr=sr), 75))

    # 6. 스펙트럼 대역폭 (IQR) - 스펙트럼 대역폭을 이용
    features.append(np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)))

    # 7. 스펙트럼 플랫니스
    features.append(np.mean(librosa.feature.spectral_flatness(y=y)))

    # 8-19. MFCC (13개의 MFCC 계수 추출)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_means = np.mean(mfccs.T, axis=0)
    features.extend(mfcc_means)

    # 추가된 값 - 스펙트럼 롤오프의 평균을 하나만 사용
    # 20. 스펙트럼 롤오프 평균
    features.append(np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)))

    return np.array(features)

# Step 6: 새로운 음성 파일을 이용해 성별 예측
def predict_voice_gender_from_file(file_path):
    new_voice_features = extract_features_from_audio(file_path)
    if len(new_voice_features) != len(X.columns):
        raise ValueError(f"특징 수가 일치하지 않습니다. 모델은 {len(X.columns)}개의 특징을 기대하지만, {len(new_voice_features)}개의 특징을 입력받았습니다.")
    
    prediction = clf.predict([new_voice_features])
    if prediction == 0:
        return "남성"
    else:
        return "여성"

# 예시로 새로운 음성 파일을 넣고 성별 예측하기
audio_file_path = r'C:\Projects\Python_basic\최종프로젝트\data\female\03_나는_다인이야.wav'  # 새로운 음성 파일 경로 (WAV 또는 MP3)
predicted_gender = predict_voice_gender_from_file(audio_file_path)
print(f"예측된 성별: {predicted_gender}")


모델의 정확도: 79.42%
예측된 성별: 남성


c:\Users\Admin\miniconda3\envs\gumi_env310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
